In [1]:
#import data

import pandas as pd
import tensorflow as tf
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"


PRED_DATA= pd.read_csv('https://s3.amazonaws.com/drivendata/data/2/public/5c9fa979-5a84-45d6-93b9-543d1a0efc41.csv',index_col=0)

data = pd.read_csv('https://s3.amazonaws.com/drivendata/data/2/public/9db113a1-cdbe-4b1c-98c2-11590f124dd8.csv',index_col=0)

submission = pd.read_csv('https://s3.amazonaws.com/drivendata/data/2/public/BloodDonationSubmissionFormat.csv',index_col=0)




C:\Users\python\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#prepare Data

from sklearn import preprocessing

def prepare_X (x):
    if 'Made Donation in March 2007' in x.columns :  # means its the training data 
        x = x.drop(['Made Donation in March 2007'],axis=1)
        
    x = preprocessing.scale(x)
    return x

def prepare_y (y):

    y.loc[: , ("Didnt")] = y['Made Donation in March 2007'] == 0
    y.loc[: , ("Didnt")] = y["Didnt"].astype(int) 
    y = y.loc[:, ['Made Donation in March 2007', 'Didnt']]
    return y


#train 
X = prepare_X(data)
y = prepare_y(data)

#prediction data
pred_data = prepare_X(PRED_DATA)

In [3]:
#split data to train and test 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=17)

In [4]:
#variables and parameters

n_hidden_1 = 32
n_input = X_train.shape[1]
n_classes = y_train.shape[1]
training_epochs = 10000
display_step = 1000
batch_size = 50
learning_rate=0.0001


In [5]:
# create placeholders
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [6]:
# weights and biases

weights = {
    'W1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}



In [7]:
## forward propagation 

def nn_forward (x):
    layer_1 = tf.add(tf.matmul(x, weights['W1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [8]:
# predictions cost/loss optimization

predictions = nn_forward (x)

cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=predictions, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1)), "float"))

In [10]:
# session 
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    

    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(X_train) / batch_size)
        x_batches = np.array_split(X_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c , acc = sess.run([optimizer, cost,accuracy], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y 
                            })
            
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: y_test}))
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)


Epoch: 0001 cost= 1.382679542
Epoch: 1001 cost= 0.462810116
Epoch: 2001 cost= 0.431867705
Epoch: 3001 cost= 0.420971887
Epoch: 4001 cost= 0.414407180
Epoch: 5001 cost= 0.408638097
Epoch: 6001 cost= 0.404351072
Epoch: 7001 cost= 0.401578304
Epoch: 8001 cost= 0.399222665
Epoch: 9001 cost= 0.397168716
Optimization Finished!
Accuracy: 0.75
Model saved in path: /tmp/model.ckpt


In [11]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    pred = sess.run(predictions , feed_dict={ x: pred_data })
 

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Model restored.


In [12]:

df = pd.DataFrame(pred[:,1] , columns = ["Made Donation in March 2007"],index=PRED_DATA.index)


df.to_csv("Blood_Tensorflow.csv")

